In [ ]:
from docx import Document as Document2
from docx.document import Document as Document3
from docx.text.paragraph import Paragraph, Run
from docx.table import _Cell, Table
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P
import xml.etree.ElementTree as ET
from xml.etree import ElementTree
import re

def getDocument_metadata(doc_name):

    document = Document2(doc_name)

    def iter_block_items(parent):
        if isinstance(parent, Document3):
            parent_elm = parent.element.body
        elif isinstance(parent, _Cell):
            parent_elm = parent._tc
        else:
            raise ValueError("Instance error.")

        for child in parent_elm.iterchildren():
            if isinstance(child, CT_P):
                yield Paragraph(child, parent)
            elif isinstance(child, CT_Tbl):
                yield Table(child, parent)

    metadata = []
    for block in iter_block_items(document):
        metadata.append(block)
    return metadata

def table_extract_str(table):
    table_str = []
    for row in table.rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                table_str.append(paragraph.text.replace(u"\xa0", u" ").replace("\uf0b7", ""))
    table_str = "\n".join(table_str)
    return table_str


def getCSRSections(document, document_metadata, doc_result, removehead=True, removeNumbers=True):
    doc_extract = doc_result.text.replace("</a>", "").replace("--\t", ". \n")
    doc_extract = doc_extract.replace("\xa0", " ").replace("\uf0b7", "")
    doc_extract = doc_extract.split("\n")

    header_list = []
    for para in document.paragraphs:
#            if para.style.name.lower().startswith('head'):
        if para.style is not None and 'head' in para.style.name.lower():
            header_list.append(para.text.replace(u"\xa0", u" ").replace("\uf0b7", "").rstrip())

    if "" in header_list:
        header_list.remove("")

    csr_sections = {}
    csr_multi_check ={}
    text_list = []
    header = ""
    for para_m in document_metadata:
        try:
            para = para_m.text.replace(u"\xa0", u" ").replace("\uf0b7", "")
        except:
            para = table_extract_str(para_m.table)
#            para = para.replace("\t"," ")
        para = para.rstrip()
        if para in header_list and 'head' in para_m.style.name.lower():
            if header not in csr_multi_check.keys():
                csr_multi_check[header] = 0
                csr_sections[header] = "\n".join(text_list)
                header = para.replace(u"\xa0", u" ").replace("\uf0b7", "")
                text_list = []
            else:
                csr_multi_check[header] += 1
                csr_sections[f"{header}_{csr_multi_check[header]}"] = "\n".join(text_list)
                header = para.replace(u"\xa0", u" ").replace("\uf0b7", "")
                text_list = []
        else:
            text_list.append(para.replace(u"\xa0", u" ").replace("\uf0b7", ""))
    #for the last header
    if header not in csr_multi_check.keys():
        csr_multi_check[header] = 0
        csr_sections[header] = "\n".join(text_list)
        header = para.replace(u"\xa0", u" ").replace("\uf0b7", "")
        text_list = []
    else:
        csr_multi_check[header] += 1
        csr_sections[f"{header}_{csr_multi_check[header]}"] = "\n".join(text_list)
        header = para.replace(u"\xa0", u" ").replace("\uf0b7", "")
        text_list = []
    csr_sections_keys = list(csr_sections.keys())

    if removehead == True:
        for sec in csr_sections_keys:
            if sec == "":
                del csr_sections[sec]

    ## removing the sections numbers if present before the section heading from csr sections
    if removeNumbers:
        try:
            csr_sections = { re.sub(r'^[0-9\.]+\s','',re.sub(r'\n','',k)).strip():v  for k,v in csr_sections.items()}
        except:
            pass
    
    ###removing the trailing spaces
    csr_sections = { k.strip():v  for k,v in csr_sections.items()}
    return csr_sections